In [1]:
pip install transformers datasets torch sentence-transformers scikit-learn seqeval pandas


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Correct base path for your dataset
DATA_DIR = r"C:\Users\satya\Downloads\Miimansa Problem\Assignment\data\CADEC.v2"


In [4]:
import glob, os

# Base dataset path
DATA_DIR = r"C:\Users\satya\Downloads\Miimansa Problem\Assignment\data\CADEC.v2"

# Look inside original folder for .ann files
files = glob.glob(os.path.join(DATA_DIR, "original", "*.ann"))
print("Total original files found:", len(files))
print("First 5 files:", files[:5])


Total original files found: 1250
First 5 files: ['C:\\Users\\satya\\Downloads\\Miimansa Problem\\Assignment\\data\\CADEC.v2\\original\\ARTHROTEC.1.ann', 'C:\\Users\\satya\\Downloads\\Miimansa Problem\\Assignment\\data\\CADEC.v2\\original\\ARTHROTEC.10.ann', 'C:\\Users\\satya\\Downloads\\Miimansa Problem\\Assignment\\data\\CADEC.v2\\original\\ARTHROTEC.100.ann', 'C:\\Users\\satya\\Downloads\\Miimansa Problem\\Assignment\\data\\CADEC.v2\\original\\ARTHROTEC.101.ann', 'C:\\Users\\satya\\Downloads\\Miimansa Problem\\Assignment\\data\\CADEC.v2\\original\\ARTHROTEC.102.ann']


In [7]:
sample_file = r"C:\Users\satya\Downloads\Miimansa Problem\Assignment\data\CADEC.v2\original\ARTHROTEC.1.ann"

with open(sample_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(i, repr(line))
        if i > 10:
            break


0 'T1\tADR 9 19\tbit drowsy\n'
1 '#1\tAnnotatorNotes T1\tDrowsy\n'
2 'T2\tADR 29 50\tlittle blurred vision\n'
3 '#2\tAnnotatorNotes T2\tBlurred Vision\n'
4 'T3\tDrug 93 102\tArthrotec\n'
5 'T5\tDisease 179 188\tarthritis\n'
6 'T6\tSymptom 260 265\tagony\n'
7 'T4\tADR 62 78\tgastric problems\n'
8 'T7\tSymptom 412 417\tpains\n'
9 'T8\tADR 437 453\tfeel a bit weird\n'
10 '#8\tAnnotatorNotes T7\tImplies a previous symptom of pain.\n'


In [8]:
with open(sample_file, "r", encoding="utf-8") as f:
    for line in f:
        print(repr(line.split("\t")))


['T1', 'ADR 9 19', 'bit drowsy\n']
['#1', 'AnnotatorNotes T1', 'Drowsy\n']
['T2', 'ADR 29 50', 'little blurred vision\n']
['#2', 'AnnotatorNotes T2', 'Blurred Vision\n']
['T3', 'Drug 93 102', 'Arthrotec\n']
['T5', 'Disease 179 188', 'arthritis\n']
['T6', 'Symptom 260 265', 'agony\n']
['T4', 'ADR 62 78', 'gastric problems\n']
['T7', 'Symptom 412 417', 'pains\n']
['T8', 'ADR 437 453', 'feel a bit weird\n']
['#8', 'AnnotatorNotes T7', 'Implies a previous symptom of pain.\n']


In [12]:
import os
from collections import defaultdict
import pandas as pd

# Paths
DATA_DIR = r"C:\Users\satya\Downloads\Miimansa Problem\Assignment\data\CADEC.v2\original"
OUTPUT_DIR = r"C:\Users\satya\Downloads\Miimansa Problem\Assignment\outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Dictionary to store unique entities for each label
distinct_entities = defaultdict(set)

# Iterate through all annotation files
for filename in os.listdir(DATA_DIR):
    if not filename.endswith(".ann"):
        continue
    filepath = os.path.join(DATA_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("#"):
                continue  # Skip comments and empty lines
            parts = line.split("\t")
            if len(parts) < 3:
                continue  # Skip malformed lines

            # Example: T1	ADR 9 19	bit drowsy
            tag = parts[0]
            label_info = parts[1].split()
            label = label_info[0]  # ADR / Drug / Disease / Symptom
            entity_text = parts[2].strip().lower()

            distinct_entities[label].add(entity_text)

# Prepare DataFrame for summary
results = {label: len(entities) for label, entities in distinct_entities.items()}
df = pd.DataFrame(list(results.items()), columns=["Entity Type", "Unique Entity Count"])

# Save to CSV
output_file = os.path.join(OUTPUT_DIR, "task1_entities_final.csv")
df.to_csv(output_file, index=False)

# Print results
print("\nEntity Counts:\n", df)
print(f"\n✅ Results saved to: {output_file}")

# Print some samples
for label in ["ADR", "Drug", "Disease", "Symptom"]:
    if label in distinct_entities:
        print(f"\nLabel: {label}")
        print(f"Total unique entities: {len(distinct_entities[label])}")
        print("Examples:", list(distinct_entities[label])[:10])
        print("-" * 40)



Entity Counts:
   Entity Type  Unique Entity Count
0         ADR                 3400
1        Drug                  323
2     Disease                  164
3     Symptom                  148
4     Finding                  298

✅ Results saved to: C:\Users\satya\Downloads\Miimansa Problem\Assignment\outputs\task1_entities_final.csv

Label: ADR
Total unique entities: 3400
Examples: ['cramps', 'bowel/uterine cramping', 'cold sweat', 'difficulty with speech', 'easy bruising', 'gastric upsets', 'loss of muscle mass in chest', 'pains in my left knee', 'raised blood pressure', 'blind spot']
----------------------------------------

Label: Drug
Total unique entities: 323
Examples: ['simvostatin', 'voltaren rapid 25', 'prednisone', 'clonidine', 'liptor', 'cortisone injection', 'prednisone 10mg', 'statins', 'celebrex 200mg', 'effexor']
----------------------------------------

Label: Disease
Total unique entities: 164
Examples: ['plantar fasciitis', 'lower back pain', 'oa', 'severe osteoarthrit